In [9]:
import gym
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [11]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [35]:
def ppo_eval(p, L, t_t, n_iter):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
    cont_env = make_vec_env('inventory_cont_config_model-v0', n_envs=8, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1)
    cont_model.learn(total_timesteps=t_t)#testing 2000

    cont_model.save("inv_cont_2")
    trained_model = PPO.load("inv_cont_2")
    env_eval = make_vec_env('inventory_cont_config_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    numiter = n_iter#test
    res_mean, res_std = ppo_evaluate(trained_model, env_eval, numiter)
    print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

    return res_mean, res_std

In [37]:
listp = [0.25,1,4,9,39,99]
listL = [1,4,10,20,30,50,70,100]
t_t = 200000
n_iter = 50000
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        res_mean, res_std = ppo_eval(p,L,t_t, n_iter)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':res_mean, 'res_std': res_std}, ignore_index=True)

Running PPO w/: p= 0.25 , L= 1
Using cpu device
------------------------------------------
| time/                   |              |
|    fps                  | 10401        |
|    iterations           | 1            |
|    time_elapsed         | 1            |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0004953145 |
|    clip_fraction        | 4.88e-05     |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.42        |
|    explained_variance   | -5.72e-05    |
|    learning_rate        | 0.0003       |
|    loss                 | 1.52e+06     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.000879    |
|    std                  | 0.996        |
|    value_loss           | 3.14e+06     |
------------------------------------------


KeyboardInterrupt: 

In [34]:
ppo_res

,p,L,res_mean,res_std
0,0.25,1.0,-0.266697,0.039537
1,0.25,4.0,-0.224051,0.016760
2,0.25,10.0,-0.257662,0.019421
3,0.25,20.0,-0.241807,0.022766
4,0.25,30.0,-0.251493,0.024331
5,0.25,50.0,-0.301303,0.041520
6,0.25,70.0,-0.253412,0.046970
7,0.25,100.0,-0.253412,0.025594
8,1.00,1.0,-0.826356,0.093646
9,1.00,4.0,-0.823574,0.063262


In [ ]:
#continuous model try with cont action

#env = make_vec_env('inventory_cont_model-v0', ContCONFIG = {'h': 1, 'p': 1, 'L': 10, 'lambda': 1}, n_envs=4)
#how to set parameters???
cont_env = make_vec_env('inventory_cont_model-v0', n_envs=8)
cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1)
cont_model.learn(total_timesteps=200000)
# fixed issue with vector, now action can be continuous

#continuous evaluation
cont_model.save("inv_cont_1")
trained_model = PPO.load("inv_cont_1")
env1 = gym.make('inventory_cont_model-v0')
numiter = 50000
res_mean, res_std = ppo_evaluate(trained_model, env1, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [ ]:
#Discrete environment
#h=1,p=99,L=1,max_inventory=100,max_action=100,binomial(6,0.5)
env = make_vec_env('inventory_model-v0', n_envs=4)
model = PPO(MlpPolicy, env, verbose=1, gamma = 1)
model.learn(total_timesteps=1000000)
model.save("inv_2")

trained_model2 = PPO.load("inv_2")
env2 = gym.make('inventory_model-v0')
numiter = 200000
res_mean, res_std = ppo_evaluate(trained_model2, env2, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 

In [ ]:
#Discrete environment 2
#h=1,p=99,L=1,max_inventory=100,max_action=6,binomial(6,0.5)
env = make_vec_env('inventory_model-v0', n_envs=4)
model = PPO(MlpPolicy, env, verbose=1, gamma = 1)
model.learn(total_timesteps=1000000)
model.save("inv_3")

#set action space to be small([0,1,2,3,4,5,6]) seems to get reasonable results...
trained_model3 = PPO.load("inv_3")
env3 = gym.make('inventory_model-v0')
numiter = 200000
res_mean, res_std = ppo_evaluate(trained_model3, env3, numiter)
print(-res_mean,'+/=',1.96*res_std/np.sqrt(5)) 